In [1]:
import requests
import json

FASTAPI_URL = "http://127.0.0.1:8000/embed/"

In [2]:
'''
def run_embedding_query_loop():
    while True:
        try:
            handle_query_embedding()
        except KeyboardInterrupt:
            return
'''        
def handle_query_embedding(sentences:list, uid:str):
    #url
    url = FASTAPI_URL

    #headers
    headers = {
        "Content-Type": "application/json"
    }

    #data
    #query = "my name is bongsoo. who are you?"
    #query = "안녕하세요.지금은 curl 테스트 입니다."
    #query = input("임베딩할 문장 입력: ")
    #sentences = ['제주도', '전라도', '서울특별시', '충청도', '독도', '경기도', '함경북도', '울릉도', '대마도', '서귀포시']
    # json 구조
    temp = {
        "uid": uid,
        "sentence": sentences
        #"text2": text2,
    }

    print(temp)
    
    # dict 을 json으로 변환
    data = json.dumps(temp)
    
    try:
        response = requests.post(url, headers=headers, data=data)
    except requests.exceptions.Timeout as errd:
        print("Timeout Error:", errd)
        return ""
    except requests.exceptions.ConnectionError as errc:
        print("Connection Error:", errc)
        return ""
    except requests.exceptions.HTTPError as errb:
        print("Http Error:", errb)
        return ""
    except requests.exceptions.RequestException as erra:#Any Error except
        print("AnyException Error:", erra)
        return ""
    
    print(type(response.text))
    #print("response.text: ", response.text)
    
    return response.text

In [3]:
import numpy as np

sentences = ['제주도', '전라도', '서울특별시', '충청도', '독도', '경기도', '함경북도', '울릉도', '강원도', '서귀포시']
uid = '대한민국지명'
response = handle_query_embedding(sentences, uid)

# str형을 dict형으로 변환 
import json
embed_dict = json.loads(response)
print(f'type(embed_dict):{type(embed_dict)}')
embeds = embed_dict['embed']
print(f'type(embeds):{type(embeds)}')


embed_list = []
for embed in embeds:
    embed = embed.split(",") # str형을 list로 변환  
    
    # list형을 array형 변환 함.
    # -반드시 ** astype('float16') 해야 함.=>아니면 ES 추가시 mapper_parsing_exception 에러남.
    # -이유는 ES dense_vector 는 반드시 숫자형(float16, float32 등)이 입력되어야 함.
    # - str 형 입력 하면 에러남. 그리고 type이 저장안되도 에러남.
    embed = np.array(embed).astype('float16')  
    embed_list.append(embed) # 리스트에 추가
   
#embed_list = np.array(embed_list).astype('float16')
#print(f'embed_list.shape: {embed_list.shape}')


{'uid': '대한민국지명', 'sentence': ['제주도', '전라도', '서울특별시', '충청도', '독도', '경기도', '함경북도', '울릉도', '강원도', '서귀포시']}
<class 'str'>
type(embed_dict):<class 'dict'>
type(embeds):<class 'list'>
type(embed): <class 'str'>
type(embed): <class 'str'>
type(embed): <class 'str'>
type(embed): <class 'str'>
type(embed): <class 'str'>
type(embed): <class 'str'>
type(embed): <class 'str'>
type(embed): <class 'str'>
type(embed): <class 'str'>
type(embed): <class 'str'>


'\nembed_list = []\nfor embed in embeds:\n    embed = embed.split(",") # str형을 list로 변환  \n    embed = np.array(embed).astype(\'float16\')  # list형을 array형 변환 하면서 반드시 astype(\'float16\') 해야 함.=>아니면 ES 추가시 에러남.\n    embed_list.append(embed) # 리스트에 추가\n   \n#embed_list = np.array(embed_list).astype(\'float16\')\n#print(f\'embed_list.shape: {embed_list.shape}\')\n'

In [4]:
embed_list

[['-0.0262,-0.013214,0.0002446,0.03406,0.1206,-0.00852,0.1345,-0.1382,-0.02455,-0.06192,-0.05585,-0.06256,-0.09155,0.0119,-0.1412,0.05096,-0.06018,-0.06158,-0.02184,-0.0239,0.014404,0.0748,-0.0314,-0.0752,0.0111,-0.08405,-0.12244,-0.1284,-0.0474,0.11365,-0.1533,0.1025,0.1871,-0.1296,0.1936,-0.092,-0.0251,-0.05444,-0.0941,0.05984,-0.0388,0.02135,-0.05032,0.01837,0.02237,-0.11914,0.02327,0.04678,0.129,0.00771,0.2189,-0.1438,0.03607,-0.0827,-0.213,-0.01418,-0.04123,-0.009514,0.0484,0.07697,-0.07965,-0.005985,0.0595,0.01065,-0.0506,-0.10626,0.0974,0.10077,0.096,0.01707,-0.03955,0.001261,-0.1203,0.03018,0.04962,-0.0405,0.09845,0.007767,-0.10144,-0.0919,-0.03111,0.1377,-0.01913,-0.01682,0.0709,-0.1603,0.0307,-0.09406,-0.1981,-0.0927,-0.0298,0.02759,0.01761,0.0622,-0.0132,-0.1251,0.2002,-0.0679,0.1544,0.0243,0.1349,0.0693,-0.03867,-0.10175,-0.00865,-0.11316,-0.0756,-0.05817,-0.10095,0.0839,-0.012764,0.03604,0.11896,-0.12134,-0.06885,-0.143,0.007214,0.188,-0.05447,-0.01344,0.0751,-0.0309,0.042

In [5]:
INDEX_FILE = './data/mpower10u_128d_10.json'
INDEX_NAME = 'fastapi_test_128d_10_2'

In [6]:
#---------------------------------------------------------------------------
# ES 인덱스 생성
# -in : es : ElasticSearch 객체.
# -in : create : 기존에 동일한 인덱스는 삭제하고 다시 생성.
#---------------------------------------------------------------------------
def create_index(es, create:bool = True):
    
    if create == True:
        es.indices.delete(index=INDEX_NAME, ignore=[404])
        count = 0
        
        # 인덱스 생성
        with open(INDEX_FILE) as index_file:
            source = index_file.read().strip()
            count += 1
            print(f'{count}:{source}') # 인덱스 구조 출력
            es.indices.create(index=INDEX_NAME, body=source)

In [7]:
#---------------------------------------------------------------------------
# 인덱스 batch 처리
# - in: ES 객체
# - in: docs=인덱스 처리할 data
# - in: vector_len=한문서에 인덱싱할 벡터수=클러스터링수와 동일(기본=10개)
# - in: dim_size=벡터 차원(기본=128)
#---------------------------------------------------------------------------
def index_batch(es, docs, vector_len:int=10, dim_size:int=128):
        
    requests = []
    
    for i, doc in enumerate(tqdm(docs)):
        rfile_name = doc['rfile_name']
        rfile_text = doc['rfile_text']
        dense_vectors = doc['dense_vectors']
        
        #--------------------------------------------------------------------
        # ES에 문단 인덱싱 처리
        request = {}  #dict 정의
        request["rfile_name"] = rfile_name       # 제목               
        request["rfile_text"] = rfile_text   # 문장
        
        request["_op_type"] = "index"        
        request["_index"] = INDEX_NAME
        
        # vector 1~40 까지 값을 0으로 초기화 해줌.
        for i in range(vector_len):
            request["vector"+str(i+1)] = np.zeros((dim_size))
            
        # vector 값들을 담음.
        for i, dense_vector in enumerate(dense_vectors):
            request["vector"+str(i+1)] = dense_vector
            
        requests.append(request)
        #--------------------------------------------------------------------
                
    # batch 단위로 한꺼번에 es에 데이터 insert 시킴     
    bulk(es, requests)

In [8]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from tqdm.notebook import tqdm

# 3. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info())

# 4. 인덱스 생성
create_index(es, True)

docs = []
doc = {} #dict 선언
doc['rfile_name'] = "test1"      # contextid 담음
doc['rfile_text'] = "섬이름"      # text 담음.
doc['dense_vectors'] = embed_list
docs.append(doc)

# 5. index 처리
index_batch(es, docs, vector_len=10, dim_size=128)

{'name': 'node-1', 'cluster_name': 'mpower', 'cluster_uuid': '2TN-XaLkReC_BmRoReVUfw', 'version': {'number': '7.17.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff', 'build_date': '2022-04-19T08:11:19.070913226Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}
1:{			"settings": {
				"number_of_shards": 2,
				"number_of_replicas": 1,
				"index": {
					"merge": {
						"scheduler": {
							"max_thread_count": 1
						}
					}
				},
				"analysis": {
					"analyzer": {
						"mpower10u_korean_analyzer": {
							"type": "custom",
							"tokenizer": "mpower10u_korean_tokenizer",
							"filter": [
								"lowercase",
								"stop",
								"nori_readingform",
								"mpower10u_korean_pos"
							]
						}
					},
					"tokenizer": {
						"mpower10u_korean_tokenizer": {
							"typ

C:\Users\A\AppData\Local\Temp\ipykernel_14212\2744547294.py:17: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=source)


  0%|          | 0/1 [00:00<?, ?it/s]

BulkIndexError: ('1 document(s) failed to index.', [{'index': {'_index': 'fastapi_test_128d_10_2', '_type': '_doc', '_id': 'aIP2m4cBEg8Lpv5fX27i', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse', 'caused_by': {'type': 'parsing_exception', 'reason': 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [VALUE_STRING]', 'line': 1, 'col': 59}}, 'data': {'rfile_name': 'test1', 'rfile_text': '섬이름', 'vector1': ['-0.0262,-0.013214,0.0002446,0.03406,0.1206,-0.00852,0.1345,-0.1382,-0.02455,-0.06192,-0.05585,-0.06256,-0.09155,0.0119,-0.1412,0.05096,-0.06018,-0.06158,-0.02184,-0.0239,0.014404,0.0748,-0.0314,-0.0752,0.0111,-0.08405,-0.12244,-0.1284,-0.0474,0.11365,-0.1533,0.1025,0.1871,-0.1296,0.1936,-0.092,-0.0251,-0.05444,-0.0941,0.05984,-0.0388,0.02135,-0.05032,0.01837,0.02237,-0.11914,0.02327,0.04678,0.129,0.00771,0.2189,-0.1438,0.03607,-0.0827,-0.213,-0.01418,-0.04123,-0.009514,0.0484,0.07697,-0.07965,-0.005985,0.0595,0.01065,-0.050